pip3 install plotly==5.24.1 
pip3 install folium==0.17.0
pip3 install mapclassify==2.8.1
pip3 install matplotlib==3.5.1 
pip3 install numpy==1.24.0
pip3 install numba==0.60.0

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from typing import Any
import pickle as pkl
import pyproj
from shapely.geometry import Polygon, LineString
from shapely.ops import split
import plotly.graph_objects as go

/home/ros/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ros/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


# Base Map Overlay Visualizer

## Initialization

In [2]:
from shapely.geometry import box, MultiPolygon, Point
from typing import Any
from custom_interfaces.msg import HelperLatLon, Path
from local_pathfinding.coord_systems import latlon_to_xy
import plotly.graph_objects as go
import numpy as np

In [21]:
sailbot_position_lat_lons = [
    HelperLatLon(latitude=49.50000, longitude=-139.922429),
    HelperLatLon(latitude=49.667979, longitude=-140.807750),
    HelperLatLon(latitude=49.979489, longitude=-141.808892),
    HelperLatLon(latitude=50.331437, longitude=-142.951804),
    HelperLatLon(latitude=50.822548, longitude=-144.336486),
]
local_waypoints_lat_lon = Path(
    waypoints=[
        HelperLatLon(latitude=49.208985, longitude=-139.922429),
        HelperLatLon(latitude=49.567979, longitude=-140.807750),
        HelperLatLon(latitude=49.979489, longitude=-141.708892),
        HelperLatLon(latitude=50.331437, longitude=-142.851804),
        HelperLatLon(latitude=50.722548, longitude=-144.236486),
    ]
)
reference_point_lat_lon = HelperLatLon(
    latitude=50.722548, longitude=-144.236486
)  # Last global waypoint

# Convert lat/lon to xy coordinates
sailbot_position_xy = [
    latlon_to_xy(reference=reference_point_lat_lon, latlon=sailbot_position_lat_lon)
    for sailbot_position_lat_lon in sailbot_position_lat_lons
]
local_waypoints_xy = [
    latlon_to_xy(reference=reference_point_lat_lon, latlon=waypoint)
    for waypoint in local_waypoints_lat_lon.waypoints
]


waypoints_lat = [waypoint.latitude for waypoint in local_waypoints_lat_lon.waypoints]
waypoints_lon = [waypoint.longitude for waypoint in local_waypoints_lat_lon.waypoints]

waypoints_x = [local_waypoints_xy[i].x for i in range(len(local_waypoints_xy))]
waypoints_y = [local_waypoints_xy[i].y for i in range(len(local_waypoints_xy))]

boat_speed = [10.0, 5, 15, 20, 9]  # Speed in knots
boat_heading = [0, 275, 280, 285, 290]  # Heading in degrees

boat_history_lat_lon = [
    {
        "position": sailbot_position_lat_lons[i],
        "waypoints_lat": waypoints_lat,
        "waypoints_lon": waypoints_lon,
        "boat_speed": boat_speed[i],
        "boat_heading": boat_heading[i],
    }
    for i in range(5)
]

boat_history_xy = [
    {
        "position": sailbot_position_xy[i],
        "waypoints_x": waypoints_x,
        "waypoints_y": waypoints_y,
        "boat_speed": boat_speed[i],
        "boat_heading": boat_heading[i],
    }
    for i in range(5)
]

print("Boat History (lat/lon):", boat_history_lat_lon)
print("Boat History (xy):", boat_history_xy)

current_position = 0

Boat History (lat/lon): [{'position': custom_interfaces.msg.HelperLatLon(latitude=49.5, longitude=-139.922429), 'waypoints_lat': [49.208985, 49.567979, 49.979489, 50.331437, 50.722548], 'waypoints_lon': [-139.922429, -140.80775, -141.708892, -142.851804, -144.236486], 'boat_speed': 10.0, 'boat_heading': 0}, {'position': custom_interfaces.msg.HelperLatLon(latitude=49.667979, longitude=-140.80775), 'waypoints_lat': [49.208985, 49.567979, 49.979489, 50.331437, 50.722548], 'waypoints_lon': [-139.922429, -140.80775, -141.708892, -142.851804, -144.236486], 'boat_speed': 5, 'boat_heading': 275}, {'position': custom_interfaces.msg.HelperLatLon(latitude=49.979489, longitude=-141.808892), 'waypoints_lat': [49.208985, 49.567979, 49.979489, 50.331437, 50.722548], 'waypoints_lon': [-139.922429, -140.80775, -141.708892, -142.851804, -144.236486], 'boat_speed': 15, 'boat_heading': 280}, {'position': custom_interfaces.msg.HelperLatLon(latitude=50.331437, longitude=-142.951804), 'waypoints_lat': [49.20

In [ ]:
def load_pkl(file_path: str) -> Any:
    with open(file_path, "rb") as f:
        return pkl.load(f)


land_obstacle = load_pkl("/workspaces/sailbot_workspace/src/local_pathfinding/land/pkl/land.pkl")
gdf = gpd.read_file(
    "/workspaces/sailbot_workspace/src/local_pathfinding/land/territorial_waters.geojson"
)


WGS84 = pyproj.CRS("EPSG:4326")
gdf_new = gpd.GeoDataFrame(geometry=[land_obstacle], crs=WGS84)
gdf_new.explore()  # Good

#

# Local Pathfinding Visualizer Using Plotly (Lat/Lon)

In [22]:
# Initial plot
center_index = len(local_waypoints_lat_lon.waypoints) // 2
initial_boat_state = go.Scattermapbox(
    lat=[boat_history_lat_lon[0]["position"].latitude],
    lon=[boat_history_lat_lon[0]["position"].longitude],
    mode="markers",
    marker=go.scattermapbox.Marker(size=14, color="red"),
    text=["Boat"],
    name="Boat",
)
initial_state = [
    go.Scattermapbox(
        lat=[waypoints_lat[0]],
        lon=[waypoints_lon[0]],
        mode="markers",
        marker=go.scattermapbox.Marker(size=14),
        text=["Start"],
        name="Start",
    ),
    go.Scattermapbox(
        lat=waypoints_lat[1 : len(local_waypoints_lat_lon.waypoints) - 1],
        lon=waypoints_lon[1 : len(local_waypoints_lat_lon.waypoints) - 1],
        mode="markers",
        marker=go.scattermapbox.Marker(size=14),
        text=["Intermediate"] * (len(local_waypoints_lat_lon.waypoints) - 2),
        name="Intermediate",
    ),
    go.Scattermapbox(
        lat=[waypoints_lat[-1]],
        lon=[waypoints_lon[-1]],
        mode="markers",
        marker=go.scattermapbox.Marker(size=14),
        text=["Goal"] * (len(local_waypoints_lat_lon.waypoints) - 2),
        name="Goal",
    ),
]
new_frames = [
    go.Frame(
        data=initial_state
        + [
            go.Scattermapbox(
                lat=[boat_history_lat_lon[i]["position"].latitude],
                lon=[boat_history_lat_lon[i]["position"].longitude],
                mode="markers",
                marker=go.scattermapbox.Marker(size=14, color="red"),
                text=["Boat"],
                name="Boat",
            )
        ],
        name=f"Boat {i}",
    )
    for i in range(0, len(boat_history_lat_lon))
]
fig = go.Figure(
    data=initial_state + [initial_boat_state],
    layout=go.Layout(
        mapbox_style="open-street-map",
        mapbox_zoom=6,
        mapbox_center={"lat": waypoints_lat[center_index], "lon": waypoints_lon[center_index]},
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        updatemenus=[
            dict(
                type="buttons",
                showactive=False,
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[
                            None,
                            {"frame": {"duration": 1000, "redraw": True}, "mode": "immediate"},
                        ],
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[
                            [None],
                            {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"},
                        ],
                    ),
                ],
            )
        ],
    ),
    frames=new_frames,
)

fig.show()

# Local Pathfinding Visualizer Using Plotly (XY)

In [24]:
# Initial plot
center_index = len(local_waypoints_xy) // 2
initial_boat_state = go.Scatter(
    x=[boat_history_xy[0]["position"].x],
    y=[boat_history_xy[0]["position"].y],
    mode="markers",
    marker=go.scatter.Marker(size=14, color="red"),
    text=["Boat"],
    name="Boat",
)
initial_state = [
    go.Scatter(
        x=[waypoints_x[0]],
        y=[waypoints_y[0]],
        mode="markers",
        marker=go.scatter.Marker(size=14),
        text=["Start"],
        name="Start",
    ),
    go.Scatter(
        x=waypoints_x[1 : len(local_waypoints_lat_lon.waypoints) - 1],
        y=waypoints_y[1 : len(local_waypoints_lat_lon.waypoints) - 1],
        mode="markers",
        marker=go.scatter.Marker(size=14),
        text=["Intermediate"] * (len(local_waypoints_lat_lon.waypoints) - 2),
        name="Intermediate",
    ),
    go.Scatter(
        x=[waypoints_x[-1]],
        y=[waypoints_y[-1]],
        mode="markers",
        marker=go.scatter.Marker(size=14),
        text=["Goal"] * (len(local_waypoints_lat_lon.waypoints) - 2),
        name="Goal",
    ),
]
new_frames = [
    go.Frame(
        data=initial_state
        + [
            go.Scatter(
                x=[boat_history_xy[i]["position"].x],
                y=[boat_history_xy[i]["position"].y],
                mode="markers",
                marker=go.scatter.Marker(size=14, color="red"),
                text=["Boat"],
                name="Boat",
            )
        ],
        name=f"Boat {i}",
    )
    for i in range(0, len(boat_history_xy))
]
fig = go.Figure(
    data=initial_state + [initial_boat_state],
    layout=go.Layout(
        mapbox_style="open-street-map",
        mapbox_zoom=7,
        mapbox_center={"lat": waypoints_x[center_index], "lon": waypoints_y[center_index]},
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        xaxis=dict(title="X"),
        yaxis=dict(title="Y"),
        updatemenus=[
            dict(
                type="buttons",
                showactive=False,
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[
                            None,
                            {"frame": {"duration": 1000, "redraw": True}, "mode": "immediate"},
                        ],
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[
                            [None],
                            {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"},
                        ],
                    ),
                ],
            )
        ],
    ),
    frames=new_frames,
)

fig.show()